In [1]:
import pandas as pd

In [2]:
# Load raw FAO data
df = pd.read_csv("data/FAOSTAT_data.csv")

# Pivot to wide format (better for analysis)
clean_df = df.pivot_table(
    index=["Area", "Item", "Year"],  # Keep these as rows
    columns="Element",              # Split metrics into columns
    values="Value",                # Use 'Value' for metrics
    aggfunc="first"                # Avoid duplicates
).reset_index()

clean_df

Element,Area,Item,Year,Area harvested,Laying,Milk Animals,Producing Animals/Slaughtered,Production,Stocks,Yield,Yield/Carcass Weight
0,Algeria,"Almonds, in shell",2020,34542.0,NaN,NaN,NaN,60832.00,NaN,1761.1,NaN
1,Algeria,"Almonds, in shell",2021,32492.0,NaN,NaN,NaN,55448.00,NaN,1706.5,NaN
2,Algeria,"Almonds, in shell",2022,38495.0,NaN,NaN,NaN,66232.31,NaN,1720.5,NaN
3,Algeria,"Almonds, in shell",2023,38544.0,NaN,NaN,NaN,69637.53,NaN,1806.7,NaN
4,Algeria,Apples,2020,33025.0,NaN,NaN,NaN,566823.50,NaN,17163.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...
15291,Zimbabwe,Blueberries,2023,506.0,NaN,NaN,NaN,2800.00,NaN,5533.6,NaN
15292,Zimbabwe,Raspberries,2020,48.0,NaN,NaN,NaN,97.69,NaN,2026.0,NaN
15293,Zimbabwe,Raspberries,2021,48.0,NaN,NaN,NaN,97.83,NaN,2041.3,NaN
15294,Zimbabwe,Raspberries,2022,48.0,NaN,NaN,NaN,98.06,NaN,2051.6,NaN


In [4]:
# Rename columns for clarity
clean_df = clean_df[["Area", "Item", "Year", "Area harvested", "Production", "Yield"]]  # Keep only needed columns
clean_df.columns = ["Country", "Crop", "Year", "Area_harvested_ha", "Production_tonnes", "Yield_kg_per_ha"]

# Save cleaned data
clean_df.to_csv("data/fao_data_cleaned.csv", index=False)

In [5]:

import numpy as np
# from faker import Faker  # For realistic fake data

# --- Load your existing dataset (40,809 records) ---
df = pd.read_csv("data/fao_data_cleaned.csv")  # Replace with your file

# --- 1. Generate Synthetic Weather Data ---
# Simulate rainfall (mm) and temperature (°C) based on region/year
def generate_weather(region, year):
    np.random.seed(hash(f"{region}{year}") % 1000)  # Deterministic randomness
    rainfall = np.random.uniform(0, 100)  # 0-100mm rainfall
    temperature = np.random.uniform(10, 35)  # 10-35°C
    return rainfall, temperature

df[["Rainfall_mm", "Temperature_C"]] = df.apply(
    lambda row: generate_weather(row["Country"], row["Year"]),
    axis=1, result_type="expand"
)

# --- 2. Generate Market Prices (USD/tonne) ---
# Base price + noise based on yield and weather
df["Price_USD_per_tonne"] = (
    500 +  # Base price
    (df["Yield_kg_per_ha"] * 0.01) +  # Higher yield → lower price
    (df["Rainfall_mm"] * -0.5)  # More rain → price drop (surplus)
).round(2)

# --- 3. Add Policy Flags (e.g., Subsidies, Export Bans) ---
# fake = Faker()
df["Policy_Flag"] = np.where(
    df["Year"] >= 2020,  # Simulate policy changes post-2020
    np.random.choice(["Subsidy", "Export Ban", "None"], p=[0.3, 0.1, 0.6]),
    "None"
)

# --- 4. Simulate Transportation Costs ---
df["Transport_Cost_USD"] = (
    np.random.uniform(10, 100, size=len(df)) *  # Base cost
    (df["Production_tonnes"] * 0.001)  # Scale with production volume
).round(2)

# --- 5. Add Demand-Supply Gap ---
df["Demand_Supply_Gap"] = (
    (df["Production_tonnes"] - np.random.uniform(0.8, 1.2, len(df)) * df["Production_tonnes"])
).round(2)



In [7]:
# --- 6. Save Enhanced Dataset ---
df.to_csv("data/enhanced_agricultural_data.csv", index=False)
print("Enhanced dataset generated with 40,809 records!")

Enhanced dataset generated with 40,809 records!


In [ ]:
#write a code to generate even number
